### Table for paper, Excess Return and Tracking Err for diff paramter values on 2 index files

In [3]:
base_path="../eit_basic_experiments/exp_3a" #Don't add '/' at start and end
T=200 #Training time period
C=1000000 #Capital available
file=1 #Index_file
m=8 #Inital Size of Kernel
lbuck=2 #Size Of Buckets
pho=0.1 #Transaction Cost Proportion
lamda=1/(1000*C) # lower_bound for capital invested in jth stock
nuh=0.8  # upper_bound
xii=1.2 # Limit for Tracking Error
k=14 #Cardinality constraint for stocks in portfolio
p=3 #If stock not selected in optimal soln in last p iters then it is to be dropped from kernel
f=12
#cj,cb are declared in script

In [4]:
import sys
sys.path.insert(0,"..") #Add root folder to sys path

from eit_basic_experiment import TestEitBasic
import time
from tqdm.notebook import tqdm
from IPython.utils import io
from joblib import Parallel, delayed
import pandas as pd

In [5]:
!rm -r ../eit_basic_experiments/exp_3a/*

zsh:1: no matches found: ../eit_basic_experiments/exp_3a/*


### Loop over files > Loop over paramter values and publish table
1. In sample Excess return/Tracking Err
2. Out of Sample

In [6]:
path_pattern=base_path+"/file={},pho={},xii={},k={},m={},lbuck={},p={}"

#### Make runs for parallel process

In [7]:
%%time
runs=[]
for file in [1]:
    for pho in [0.1,0.2,0.5]:
        for xii in [1.2,1.3,1.4]:
            for k in [10,16,25]:
                for m in [8,12,16]:
                    output=path_pattern.format(file,pho,xii,k,m,lbuck,p)
                    test_eit=TestEitBasic(output=output,T=T,C=C,file=file,m=m,lbuck=lbuck,\
                                        pho=pho,lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f)
                    runs.append(test_eit)

CPU times: user 653 µs, sys: 62 µs, total: 715 µs
Wall time: 753 µs


In [71]:
def parallel_func(eit_object):
    params=eit_object.give_params()
    result_df=eit_object.run_experiment(from_root=False,verbose=False);
    result_df["z_linear"]=eit_object.objective_linear
    step_dict={}
    step_dict[params]=result_df
    #result_dict.update(step_dict)
    return (step_dict)

In [72]:
%%time
#test_1={}
test_1=Parallel(verbose = 2,n_jobs  = -1\
            )(delayed( parallel_func )(run) for run in tqdm(runs[:16]))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:  1.1min remaining:   40.4s


CPU times: user 320 ms, sys: 49.3 ms, total: 369 ms
Wall time: 1min 24s


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  1.4min finished


In [73]:
len(test_1)

16

In [75]:
%%time
#test_2={}
test_2=Parallel(verbose = 2,n_jobs  = 1\
            )(delayed( parralel_func )(run) for run in tqdm(runs[:16]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ashish1610dhiman/opt/anaconda3/envs/eit_paper/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
../src/sort_and_buckets.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_optimal.sort_values(by="value",ascending=False,inplace=True)


Solving EIT(kernel)
***************************************************
***************************************************

Optimisation Status=0,Objective Value=1300.8276541096848
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)


For bucket=1
Solving EIT(kernel+bucket-1)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1479.0305487388496
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_15
security_29
***************************************************
Updating Lower Bound
Updating Kernel
Length of Old Kernel=8
Length of Updated Kernel=10
Current Length Kernel=10


For bucket=2
Solving EIT(kernel+bucket-2)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1530.4303577933824
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_8
*****************

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.2s remaining:    0.0s
/Users/ashish1610dhiman/opt/anaconda3/envs/eit_paper/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
../src/sort_and_buckets.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_optimal.sort_values(by="value",ascending=False,inplace=True)


Solving EIT(kernel)
***************************************************
***************************************************

Optimisation Status=0,Objective Value=1530.4303577933824
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)


For bucket=1
Solving EIT(kernel+bucket-1)
***************************************************
Optimisation Status=OptimizationStatus.INFEASIBLE,Objective Value=None
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
***************************************************


For bucket=2
Solving EIT(kernel+bucket-2)
***************************************************
Optimisation Status=OptimizationStatus.INFEASIBLE,Objective Value=None
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
***************************************************


For bucket=3
Solving EIT(kernel+bucket-3)
***************************************************
Optimisation Status=OptimizationStatus.

/Users/ashish1610dhiman/opt/anaconda3/envs/eit_paper/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
../src/sort_and_buckets.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_optimal.sort_values(by="value",ascending=False,inplace=True)


...

/Users/ashish1610dhiman/opt/anaconda3/envs/eit_paper/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
../src/sort_and_buckets.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_optimal.sort_values(by="value",ascending=False,inplace=True)


Solving EIT(kernel)
***************************************************
***************************************************

Optimisation Status=0,Objective Value=1729.269824728056
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)


For bucket=1
Solving EIT(kernel+bucket-1)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1729.4434457913067
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_1
***************************************************
Updating Lower Bound
Updating Kernel
Length of Old Kernel=16
Length of Updated Kernel=17
Current Length Kernel=17


For bucket=2
Solving EIT(kernel+bucket-2)
***************************************************
Optimisation Status=OptimizationStatus.INFEASIBLE,Objective Value=None
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
***************************************************


/Users/ashish1610dhiman/opt/anaconda3/envs/eit_paper/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
../src/sort_and_buckets.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_optimal.sort_values(by="value",ascending=False,inplace=True)


Solving EIT(kernel)
***************************************************
***************************************************

Optimisation Status=0,Objective Value=1458.985033177637
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)


For bucket=1
Solving EIT(kernel+bucket-1)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1581.5999811289403
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_15
security_29
***************************************************
Updating Lower Bound
Updating Kernel
Length of Old Kernel=8
Length of Updated Kernel=10
Current Length Kernel=10


For bucket=2
Solving EIT(kernel+bucket-2)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1672.808427850212
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_23
security_8
*******

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  4.4min finished


In [76]:
print(len(test_1),len(test_2))

16 16


In [78]:
test_2;

In [82]:
for a,b in zip(test_1,test_2):
    print(list(a.values())[0].sum().sum(),list(b.values())[0].sum().sum())

-70784347.15626384 -70784347.15626377
-5360341.466343786 -5360341.466343875
-5363655.505484827 -5363655.50548505
-189727316.84848037 -189727316.84848022
-124482402.72371905 -124482402.72371894
-46963770.01360995 -46963770.01360973
-229736037.467933 -229736037.467933
-164312042.77801237 -164312042.77801237
-86972490.63306251 -86972490.63306251
-76438345.81515083 -76438345.81515072
-10399349.162124606 -10399349.162124561
-10402823.234115656 -10402823.234115537
-192089589.69262296 -192089589.69262236
-126050604.03959657 -126050604.03959599
-47646648.89955453 -47646648.89955478
-231456272.3742422 -231456272.3742422


In [12]:
final_result_dict={}
for step_dict in result_dict:
    final_result_dict.update(step_dict)

In [13]:
final_result=pd.DataFrame()
for params,result_df in final_result_dict.items():
    param_part=pd.DataFrame([params])
    res_part=pd.DataFrame([result_df.loc[result_df.z_value.idxmax()]])
    temp=pd.concat([param_part,res_part.reset_index()],axis=1,ignore_index=False)
    final_result=final_result.append(temp)

In [14]:
final_result.to_csv(base_path+"/exp_3_results_table_1.csv")

In [16]:
final_result.columns

Index(['output', 'T', 'C', 'file', 'm', 'lbuck', 'pho', 'lamda', 'nuh', 'xii',
       'k', 'p', 'f', 'index', 'bucket', 'kernel_size', 'problem_status',
       'z_value', 'in_excess_return', 'in_tr_err', 'out_excess_return',
       'out_tr_err', 'z_linear'],
      dtype='object')

In [15]:
final_result.groupby(["k","pho"])[["z_value","z_linear","bucket"]].mean()

z_value     z_linear    bucket
k  pho                                    
10 0.1  1633.193709  1732.751987  0.666667
   0.2  1633.193709  1732.751987  0.666667
   0.5  1633.193709  1732.751987  0.666667
16 0.1  1729.275617  1732.751987  2.666667
   0.2  1729.275617  1732.751987  2.666667
   0.5  1729.275617  1732.751987  2.666667
25 0.1  1732.751987  1732.751987  4.000000
   0.2  1732.751987  1732.751987  4.000000
   0.5  1732.751987  1732.751987  4.000000

In [22]:
final_result.groupby(["k","m"])[["z_value","z_linear","bucket"]].mean()

z_value     z_linear    bucket
k  m                                     
10 8   1633.193709  1732.751987  2.000000
   12  1633.193709  1732.751987  0.000000
   16  1633.193709  1732.751987  0.000000
16 8   1729.275617  1732.751987  4.666667
   12  1729.275617  1732.751987  2.666667
   16  1729.275617  1732.751987  0.666667
25 8   1732.751987  1732.751987  6.000000
   12  1732.751987  1732.751987  4.000000
   16  1732.751987  1732.751987  2.000000

In [24]:
final_result.groupby(["k","xii"])[['in_excess_return', 'in_tr_err', 'out_excess_return',
       'out_tr_err',"bucket"]].mean()

in_excess_return      in_tr_err  out_excess_return    out_tr_err  \
k  xii                                                                     
10 1.2       1530.430358  224172.993279        3613.340702 -1.520349e+07   
   1.3       1637.074329  216741.760970        8291.844229 -2.024084e+07   
   1.4       1732.076441  261853.287566        8918.196357 -2.093902e+07   
16 1.2       1648.811498   85576.224460        6463.858074 -3.787245e+07   
   1.3       1729.443446  134887.449248        6857.818146 -3.817990e+07   
   1.4       1809.571907  126906.369317        7365.981180 -2.936720e+07   
25 1.2       1653.519570  106142.643581        7406.139759 -3.902740e+07   
   1.3       1733.535995  102298.539670        7258.858578 -3.885810e+07   
   1.4       1811.200395  147940.860604        7850.030115 -3.965217e+07   

          bucket  
k  xii            
10 1.2  0.666667  
   1.3  0.666667  
   1.4  0.666667  
16 1.2  3.000000  
   1.3  3.000000  
   1.4  2.000000  
25 1.2  4.000000  
   1.3  4.000000  
   1.4  4.000000